In [2]:
import pandas as pd
import darknet
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def save_boxImage(imagePath, boxes, path):
    cfg = './cfg/temp0922/yolov3-temp0925.cfg'
    weights = '/Users/suzaki/Sotuken/face_picture_and_tempmap/darknet/cfg/temp0922/backup/yolov3-temp0925_33000.weights'
    data = './cfg/temp0922/datasets.data'
    
    img = cv2.cvtColor(cv2.imread(imagePath), cv2.COLOR_BGR2RGB)
    for b in boxes:
        box = b[2]
        if b[1] > 0.8:
            center = (int(box[0]), int(box[1]))
            width = int(box[2])
            height = int(box[3])

            top_left = (center[0]-(width//2), center[1]-(height//2))
            btm_right = (center[0]+(width//2), center[1]+(height//2))

        #     top_left = (int(box[0]), int(box[1]))
        #     btm_right = (int(box[0]+box[2]), int(box[1]+box[3]))
            img = cv2.rectangle(img,top_left,btm_right,(0,255,0),1)
        cv2.imwrite(path, img)

def compute_mean_iou(y_pred, y_true):
    mean_iou = []
    for i in range(y_pred.shape[0]):
        area_a = (y_pred[i][2] - y_pred[i][0]) * (y_pred[i][3] - y_pred[i][1])
        area_b = (y_true[i][2] - y_true[i][0]) * (y_true[i][3] - y_true[i][1])
        iou_x1 = np.maximum(y_pred[i][0], y_true[i][0])
        iou_y1 = np.maximum(y_pred[i][1], y_true[i][1])
        iou_x2 = np.minimum(y_pred[i][2], y_true[i][2])
        iou_y2 = np.minimum(y_pred[i][3], y_true[i][3])
        iou_w = iou_x2 - iou_x1
        iou_h = iou_y2 - iou_y1
        if iou_w < 0 or iou_h < 0:
            mean_iou.append(0.0)
        else: 
            area_iou = iou_w * iou_h
            iou = area_iou / (area_a + area_b - area_iou)
            mean_iou.append(iou)
    return mean_iou

# [x1, y1, x2, y2]
a = np.array(([[0, 0, 1, 3],[0, 0, 1, 3]]), dtype=np.float32)

b = np.array(([[0, 0, 1, 1],[0, 0, 1, 1]]), dtype=np.float32)
print(compute_mean_iou(a, b))

[0.33333334, 0.33333334]


In [3]:
answer_df = pd.read_csv('./csv/annotation.csv')
print(len(answer_df))
answer_df.head()

1148


,filename,class,box1,box2,box3,box4
0,1598605729,15,0.414062,0.515625,0.484375,0.500000
1,1598606220,15,0.703125,0.695312,0.437500,0.609375
2,1598609675,15,0.640625,0.625000,0.531250,0.531250
3,1598606234,15,0.687500,0.695312,0.531250,0.609375
4,1598607102,15,0.382812,0.710938,0.484375,0.515625


In [4]:
answer_df["ans"] = answer_df.apply(lambda t: (t.box1, t.box2, t.box3, t.box4), axis=1)
answer_df.head()

,filename,class,box1,box2,box3,box4,ans
0,1598605729,15,0.414062,0.515625,0.484375,0.500000,"(0.414062, 0.515625, 0.484375, 0.5)"
1,1598606220,15,0.703125,0.695312,0.437500,0.609375,"(0.703125, 0.695312, 0.4375, 0.609375)"
2,1598609675,15,0.640625,0.625000,0.531250,0.531250,"(0.640625, 0.625, 0.53125, 0.53125)"
3,1598606234,15,0.687500,0.695312,0.531250,0.609375,"(0.6875, 0.695312, 0.53125, 0.609375)"
4,1598607102,15,0.382812,0.710938,0.484375,0.515625,"(0.382812, 0.710938, 0.484375, 0.515625)"


In [5]:
test_dir = './cfg/temp0922/kari/dataset_detection/images/'
testpath = './cfg/temp0922/temp0925/test.txt'
test_list = []
with open(testpath) as f:
    data = f.read().splitlines()
    test_list = [d.split("/")[-1].split('.jpg')[0] for d in data ]
    
print(len(test_list), test_list[0])

115 1598605200


In [6]:
answer_df2 = answer_df[answer_df["filename"].isin(test_list)].reset_index(drop=True)
print(len(answer_df2))
answer_df2.head()


115


,filename,class,box1,box2,box3,box4,ans
0,1598605729,15,0.414062,0.515625,0.484375,0.500000,"(0.414062, 0.515625, 0.484375, 0.5)"
1,1598607664,15,0.726562,0.617188,0.515625,0.546875,"(0.726562, 0.617188, 0.515625, 0.546875)"
2,1598607116,15,0.695312,0.671875,0.515625,0.531250,"(0.695312, 0.671875, 0.515625, 0.53125)"
3,1598608755,15,0.593750,0.656250,0.531250,0.562500,"(0.59375, 0.65625, 0.53125, 0.5625)"
4,1598608741,15,0.445312,0.640625,0.609375,0.593750,"(0.445312, 0.640625, 0.609375, 0.59375)"


In [7]:
cfg = './cfg/temp0922/yolov3-temp0925.cfg'
weights = '/Users/suzaki/Sotuken/face_picture_and_tempmap/darknet/cfg/temp0922/backup/yolov3-temp0925_33000.weights'
data = './cfg/temp0922/datasets.data'

bbox_list = []
i = 0
for test in test_list:
    print(i, test)
    testPath = test_dir+test+'.jpg'
    boxes = darknet.detect_box(cfg.encode(), weights.encode(), data.encode(), testPath.encode())
    boxes.insert(0, test)
#     print(boxes)
    bbox_list.append(boxes)
    i += 1
print(bbox_list[:5])

0 1598605200
1 1598609488
2 1598607666
3 1598606199
4 1598607773
5 1598607059
6 1598607057
7 1598606683
8 1598609479
9 1598610028
10 1598608333
11 1598606704
12 1598609507
13 1598607199
14 1598607056
15 1598605729
16 1598610480
17 1598608357
18 1598607766
19 1598609529
20 1598609718
21 1598608764
22 1598610088
23 1598608526
24 1598610087
25 1598609632
26 1598607794
27 1598603036
28 1596442968
29 1598609794
30 1598607657
31 1598606650
32 1598607641
33 1598610058
34 1598609505
35 1598606216
36 1598607116
37 1598610519
38 1598608515
39 1598607733
40 1598609682
41 1598605121
42 1598608755
43 1598609714
44 1598607851
45 1598608740
46 1598608815
47 1596438544
48 1598607768
49 1598609730
50 1598607664
51 1598605153
52 1598608608
53 1598606211
54 1598607167
55 1598605770
56 1598610562
57 1598608834
58 1598609484
59 1598610498
60 1596442965
61 1598608408
62 1598609493
63 1598608724
64 1598605089
65 1598608741
66 1598606698
67 1598605193
68 1598610586
69 1598605700
70 1598610524
71 1598608397
72

In [8]:
bbox_list2 = []
for idx, b in enumerate(bbox_list):
    name = b[0]
    t = []
    t.append(int(name))
    for b2 in b[1:]:
        t.append(b2[2])
    bbox_list2.append(t)
bbox_list2[0]

[1598605200,
 (41.480587005615234,
  49.36545944213867,
  35.933937072753906,
  32.53916931152344)]

In [1]:
img= cv2.cvtColor(cv2.imread(test_dir+test_list[0]+'.jpg'), cv2.COLOR_BGR2RGB)
print(test_list[0])
box = answer_df2[answer_df2["filename"] == int(test_list[0])].ans
box = box.iloc[0]
print(box)
box = (box[0]*64, box[1]*64, box[2]*64, box[3]*64)
print(box)

center = (int(box[0]), int(box[1]))
width = int(box[2])
height = int(box[3])

top_left = (center[0]-(width//2), center[1]-(height//2))
btm_right = (center[0]+(width//2), center[1]+(height//2))

img = cv2.rectangle(img,top_left,btm_right,(255, 0,0),1)

box = bbox_list2[0][1]
center = (int(float(box[0])), int(float(box[1])))
width = int(box[2])
height = int(box[3])

top_left = (center[0]-(width//2), center[1]-(height//2))
btm_right = (center[0]+(width//2), center[1]+(height//2))

img = cv2.rectangle(img,top_left,btm_right,(0,255,0),1)
plt.imshow(img)

NameError: name 'cv2' is not defined

In [10]:
est_df = pd.DataFrame(bbox_list2, columns=['name', 'est_box'])
print(len(est_df))
est_df.head()

115


,name,est_box
0,1598605200,"(41.480587005615234, 49.36545944213867, 35.933..."
1,1598609488,"(29.639713287353516, 36.30083465576172, 45.975..."
2,1598607666,"(44.14649200439453, 36.60374450683594, 34.6499..."
3,1598606199,"(19.211101531982422, 45.7623405456543, 34.7732..."
4,1598607773,"(29.86305809020996, 37.13081359863281, 36.2509..."


In [11]:
est_df['est_bbox'] = est_df.est_box.map(lambda t: (t[0]-(t[2]/2), t[1]-(t[3]/2), t[0]+(t[2]/2), t[1]+(t[3]/2)))
est_df.head()

,name,est_box,est_bbox
0,1598605200,"(41.480587005615234, 49.36545944213867, 35.933...","(23.51361846923828, 33.09587478637695, 59.4475..."
1,1598609488,"(29.639713287353516, 36.30083465576172, 45.975...","(6.651727676391602, 13.94894790649414, 52.6276..."
2,1598607666,"(44.14649200439453, 36.60374450683594, 34.6499...","(26.82152557373047, 18.724903106689453, 61.471..."
3,1598606199,"(19.211101531982422, 45.7623405456543, 34.7732...","(1.8244571685791016, 28.041135787963867, 36.59..."
4,1598607773,"(29.86305809020996, 37.13081359863281, 36.2509...","(11.737565994262695, 16.246049880981445, 47.98..."


In [12]:
answer_df2['ans_bbox'] = answer_df2.ans.map(lambda t: (t[0]*64-(t[2]*64/2), t[1]*64-(t[3]*64/2), t[0]*64+(t[2]*64/2), t[1]*64+(t[3]*64/2)))

answer_df2.head()

,filename,class,box1,box2,box3,box4,ans,ans_bbox
0,1598605729,15,0.414062,0.515625,0.484375,0.500000,"(0.414062, 0.515625, 0.484375, 0.5)","(10.999967999999999, 17.0, 41.999967999999996,..."
1,1598607664,15,0.726562,0.617188,0.515625,0.546875,"(0.726562, 0.617188, 0.515625, 0.546875)","(29.999968000000003, 22.000031999999997, 62.99..."
2,1598607116,15,0.695312,0.671875,0.515625,0.531250,"(0.695312, 0.671875, 0.515625, 0.53125)","(27.999968000000003, 26.0, 60.999968, 60.0)"
3,1598608755,15,0.593750,0.656250,0.531250,0.562500,"(0.59375, 0.65625, 0.53125, 0.5625)","(21.0, 24.0, 55.0, 60.0)"
4,1598608741,15,0.445312,0.640625,0.609375,0.593750,"(0.445312, 0.640625, 0.609375, 0.59375)","(8.999967999999999, 22.0, 47.999967999999996, ..."


In [13]:
answer_df3 = pd.merge(answer_df2, est_df, left_on='filename', right_on='name', how='left')
answer_df3.head()

,filename,class,box1,box2,box3,box4,ans,ans_bbox,name,est_box,est_bbox
0,1598605729,15,0.414062,0.515625,0.484375,0.500000,"(0.414062, 0.515625, 0.484375, 0.5)","(10.999967999999999, 17.0, 41.999967999999996,...",1598605729,"(27.644622802734375, 37.06404495239258, 35.521...","(9.883638381958008, 16.017940521240234, 45.405..."
1,1598607664,15,0.726562,0.617188,0.515625,0.546875,"(0.726562, 0.617188, 0.515625, 0.546875)","(29.999968000000003, 22.000031999999997, 62.99...",1598607664,"(41.5145378112793, 42.703243255615234, 32.1827...","(25.423147201538086, 26.17086410522461, 57.605..."
2,1598607116,15,0.695312,0.671875,0.515625,0.531250,"(0.695312, 0.671875, 0.515625, 0.53125)","(27.999968000000003, 26.0, 60.999968, 60.0)",1598607116,"(43.846946716308594, 45.181331634521484, 35.34...","(26.17422103881836, 27.35884666442871, 61.5196..."
3,1598608755,15,0.593750,0.656250,0.531250,0.562500,"(0.59375, 0.65625, 0.53125, 0.5625)","(21.0, 24.0, 55.0, 60.0)",1598608755,"(40.283042907714844, 43.36371612548828, 38.991...","(20.787311553955078, 23.460676193237305, 59.77..."
4,1598608741,15,0.445312,0.640625,0.609375,0.593750,"(0.445312, 0.640625, 0.609375, 0.59375)","(8.999967999999999, 22.0, 47.999967999999996, ...",1598608741,"(26.175922393798828, 43.032264709472656, 38.93...","(6.706882476806641, 22.904874801635742, 45.644..."


In [14]:
answer_df3["iou"] = compute_mean_iou(np.array(answer_df3.ans_bbox), np.array(answer_df3.est_bbox))
answer_df3.head()

,filename,class,box1,box2,box3,box4,ans,ans_bbox,name,est_box,est_bbox,iou
0,1598605729,15,0.414062,0.515625,0.484375,0.500000,"(0.414062, 0.515625, 0.484375, 0.5)","(10.999967999999999, 17.0, 41.999967999999996,...",1598605729,"(27.644622802734375, 37.06404495239258, 35.521...","(9.883638381958008, 16.017940521240234, 45.405...",0.663457
1,1598607664,15,0.726562,0.617188,0.515625,0.546875,"(0.726562, 0.617188, 0.515625, 0.546875)","(29.999968000000003, 22.000031999999997, 62.99...",1598607664,"(41.5145378112793, 42.703243255615234, 32.1827...","(25.423147201538086, 26.17086410522461, 57.605...",0.622105
2,1598607116,15,0.695312,0.671875,0.515625,0.531250,"(0.695312, 0.671875, 0.515625, 0.53125)","(27.999968000000003, 26.0, 60.999968, 60.0)",1598607116,"(43.846946716308594, 45.181331634521484, 35.34...","(26.17422103881836, 27.35884666442871, 61.5196...",0.825580
3,1598608755,15,0.593750,0.656250,0.531250,0.562500,"(0.59375, 0.65625, 0.53125, 0.5625)","(21.0, 24.0, 55.0, 60.0)",1598608755,"(40.283042907714844, 43.36371612548828, 38.991...","(20.787311553955078, 23.460676193237305, 59.77...",0.788610
4,1598608741,15,0.445312,0.640625,0.609375,0.593750,"(0.445312, 0.640625, 0.609375, 0.59375)","(8.999967999999999, 22.0, 47.999967999999996, ...",1598608741,"(26.175922393798828, 43.032264709472656, 38.93...","(6.706882476806641, 22.904874801635742, 45.644...",0.804305


In [15]:
answer_df3.to_csv('./csv/yolov3_detection1004.csv')